In [37]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy  as np
import pandas as pd
from scipy import stats
from statsmodels.graphics.gofplots import qqplot
from sklearn import linear_model, metrics
plt.style.use('ggplot')
%matplotlib inline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold


In [38]:
raw = pd.read_csv("./house-prices-advanced-regression-techniques/cleandata/s2_clean_dummified.csv")

In [39]:
raw.shape

(1458, 145)

In [40]:
pd.set_option('display.max_columns', 150)
raw.head()

,Id,LotFrontage,LotArea,OverallQual,TotalBsmtSF,GrLivArea,TotRmsAbvGrd,GarageCars,WoodDeckSF,OpenPorchSF,ScreenPorch,PoolQC,MiscVal,Age,Re_Age,TotalFullBath,TotalHalfBath,Artery,Feedr,PosA,PosN,RRAe,RRAn,RRNe,RRNn,LogSalePrice,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotShape_IR2,LotShape_IR3,LotShape_Reg,PavedDrive_Y,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,SeasonSold_Spring,SeasonSold_Summer,SeasonSold_Winter,Functional_Typ,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,LandSlope_Mod,LandSlope_Sev,Street_Pave,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,CentralAir_Y,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,65.0,8450,7,856,1710,8,2,0,61,0,0,0,5,5,3,1,0,0,0,0,0,0,0,0,12.247694,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0
1,2,80.0,9600,6,1262,1262,6,2,298,0,0,0,0,31,31,2,1,0,1,0,0,0,0,0,0,12.109011,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0
2,3,68.0,11250,7,920,1786,6,2,0,42,0,0,0,7,6,3,1,0,0,0,0,0,0,0,0,12.317167,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0
3,4,60.0,9550,7,756,1717,7,3,0,35,0,0,0,91,36,2,0,0,0,0,0,0,0,0,0,11.849398,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0
4,5,84.0,14260,8,1145,2198,9,3,192,84,0,0,0,8,8,3,1,0,0,0,0,0,0,0,0,12.429216,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0


In [41]:
sale_price = raw['LogSalePrice']
raw = raw.drop(['LogSalePrice','Id'],axis=1)

In [42]:
#Remove columns from train that are not in test data rather than making them 0 in test
raw = raw.drop(['Exterior1st_ImStucc', 'Exterior1st_Stone', 'HouseStyle_2.5Fin'] ,axis=1)


In [43]:
X = raw.copy()
Y = sale_price.copy()


In [44]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
ridge = Ridge()
lasso = Lasso()
net   = ElasticNet()

In [45]:
#running ridge with alpha 0 (MLR)
ridge.set_params(alpha = 0, normalize = True)
ridge.fit(X, Y)
ridge.score(X, Y)

0.9300159162360475

In [46]:
mse = Y - ridge.predict(X)
mse = mse ** 2
mse = sum(mse)
mse = mse/len(Y)
mse = np.sqrt(mse)
mse
#np.sqrt(np.sum(((Y - ridge.predict(X)))**2)/len(Y))

0.10570646793953294

In [47]:
metrics.mean_squared_error(Y, ridge.predict(X))

0.011173857364251496

In [48]:
alphas_ridge = np.linspace(0,10,50)
tuned_parameters_r = [{'alpha': alphas_ridge}]
n_folds = 5
cv = KFold(n_splits=n_folds, shuffle=True)


In [49]:
tune_ridge = GridSearchCV(ridge, tuned_parameters_r, 
                          cv=cv, refit=True, return_train_score = True, 
                          scoring = 'neg_mean_squared_error')


In [50]:
tune_ridge.fit(X,Y)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=Ridge(alpha=0, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
   random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': array([ 0.     ,  0.20408,  0.40816,  0.61224,  0.81633,  1.02041,
        1.22449,  1.42857,  1.63265,  1.83673,  2.04082,  2.2449 ,
        2.44898,  2.65306,  2.85714,  3.06122,  3.26531,  3.46939,
        3.67347,  3.87755,  4.08163,  4.28571,  4.4898 ,  4.69388,
        4....35,
        8.57143,  8.77551,  8.97959,  9.18367,  9.38776,  9.59184,
        9.79592, 10.     ])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [51]:
print(tune_ridge.best_params_)
print(np.max(tune_ridge.cv_results_['mean_test_score']))
print(np.min(tune_ridge.cv_results_['mean_test_score']))

{'alpha': 0.20408163265306123}
-0.015948628841038506
-1.0879706397559298e+22


In [52]:
#running ridge with CV (0.204) alpha 
ridge.set_params(alpha = 0.204081632, normalize = True)
ridge.fit(X, Y)
ridge.score(X, Y)

0.9195114466531388

In [53]:
#fitting lasso with alpha 0 (MLR)
lasso.set_params(alpha = 0, normalize = True, max_iter = 10000)
lasso.fit(X, Y)
lasso.score(X, Y)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:478: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.9300345228143976

In [54]:
alphas_lasso = np.logspace(-7, -2, 50)
tuned_parameters_l = [{'alpha': alphas_lasso}]
n_folds = 5
cv = KFold(n_splits=n_folds, shuffle=True)


In [55]:
tune_lasso = GridSearchCV(lasso, tuned_parameters_l, 
                          cv=cv, refit=True, return_train_score = True, 
                          scoring = 'neg_mean_squared_error')

In [56]:
tune_lasso.fit(X,Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=Lasso(alpha=0, copy_X=True, fit_intercept=True, max_iter=10000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': array([1.00000e-07, 1.26486e-07, 1.59986e-07, 2.02359e-07, 2.55955e-07,
       3.23746e-07, 4.09492e-07, 5.17947e-07, 6.55129e-07, 8.28643e-07,
       1.04811e-06, 1.32571e-06, 1.67683e-06, 2.12095e-06, 2.68270e-06,
       3.39322e-06, 4.29193e-06, 5.42868e-06, 6.86649e-06, 8.6...2.44205e-03, 3.08884e-03,
       3.90694e-03, 4.94171e-03, 6.25055e-03, 7.90604e-03, 1.00000e-02])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [57]:
print(tune_lasso.best_params_)
#print(np.max(-tune_lasso.cv_results_['mean_test_score']))
print(np.min(-tune_lasso.cv_results_['mean_test_score']))


{'alpha': 3.5564803062231284e-05}
0.015032156792191718


In [68]:
#fitting lasso with CV alpha (2.811e-05)
lasso.set_params(alpha = 3.5564803062231284e-05, normalize = True, max_iter = 10000)
lasso.fit(X, Y)
lasso.score(X, Y)


0.9251711563008882

In [69]:
raw_test = pd.read_csv(".//house-prices-advanced-regression-techniques/cleandata/s2_clean_dummified_test.csv")

In [70]:
test_IDs = raw_test['Id']
raw_test.drop(['Id', 'Exterior1st_Other'], axis = 1, inplace = True)

In [71]:
## running lasso to predict housing values 

predict_lasso = lasso.predict(raw_test)

In [72]:
predict_lasso = np.exp(predict_lasso)

In [73]:
predict_lasso

array([115146.57463134, 149339.0681931 , 170599.53786659, ...,
       154395.3781221 , 119444.07967065, 222669.7927567 ])

In [74]:
predict_lasso1 = pd.DataFrame(predict_lasso)

In [75]:
test_IDs = pd.DataFrame(test_IDs)

In [76]:
full_pred = pd.concat([test_IDs, predict_lasso1], axis = 1)
full_pred.columns = ['Id', 'SalePrice']

In [77]:
full_pred

,Id,SalePrice
0,1461,115146.574631
1,1462,149339.068193
2,1463,170599.537867
3,1464,187343.630097
4,1465,199684.811544
5,1466,176213.521434
6,1467,175175.543659
7,1468,167036.037042
8,1469,198776.666369
9,1470,121087.051979


In [78]:
full_pred.to_csv('remove cols in train not in test lasso.csv', index=False)